In [ ]:
# 소켓을 사용하기 위해서는 socket을 import해야 한다.
import socket, threading

# 분석
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle


# binder함수는 서버에서 accept가 되면 생성되는 socket 인스턴스를 통해 client로 부터 데이터를 받으면 echo형태로 재송신하는 메소드이다.
def binder(client_socket, addr):
    print('Connected by', addr)
    
    try :
        while True:
            data = client_socket.recv(4)
            length = int.from_bytes(data, "little")
            data = client_socket.recv(length)
            msg = data.decode()
            print('Received from', addr, msg)

            okt = Okt()
            stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
            max_len = 30
            
            with open('tokenizer.pickle', 'rb') as handle:
                tokenizer = pickle.load(handle)
                
            def sentiment_predict(msg):
                model = keras.models.load_model('best_model.h5')
                print("메시지 확인", msg)
                
                msg = okt.morphs(msg, stem=True)  # 토큰화
                msg = [word for word in msg if not word in stopwords]  # 불용어 제거
                encoded = tokenizer.texts_to_sequences([msg])  # 정수 인코딩
                pad_new = pad_sequences(encoded, maxlen=max_len)  # 패딩
                score = float(model.predict(pad_new))  # 예측
                result = ""
                if (score > 0.5):
                    result = "긍정 {}% 입니다".format((round(score * 100)))
                    print(result)
                else:
                    result = "부정 {}% 입니다".format((round(1-score * 100)))
                    print(result)
                    
                print(result)
                return result
            
            
            msg = sentiment_predict(msg)
            print(msg)

            data = msg.encode()
            # 바이너리의 데이터 사이즈를 구한다.
            length = len(data)
            # 데이터 사이즈를 little 엔디언 형식으로 byte로 변환한 다음 전송한다.
            client_socket.sendall(length.to_bytes(4, byteorder='little'))
            # 데이터를 클라이언트로 전송한다.
            client_socket.sendall(data)
            
    except:
        # 접속이 끊기면 except가 발생한다.
        print("except : " , addr)
    finally:
        # 접속이 끊기면 socket 리소스를 닫는다.
        client_socket.close()
            
# 소켓을 만든다.
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 소켓 레벨과 데이터 형태를 설정한다.
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
# 서버는 복수 ip를 사용하는 pc의 경우는 ip를 지정하고 그렇지 않으면 None이 아닌 ''로 설정한다.
# 포트는 pc내에서 비어있는 포트를 사용한다. cmd에서 netstat -an | find "LISTEN"으로 확인할 수 있다.
server_socket.bind(('', 9000))
# server 설정이 완료되면 listen를 시작한다.
server_socket.listen()
 
try:
  # 서버는 여러 클라이언트를 상대하기 때문에 무한 루프를 사용한다.
  while True:
    # client로 접속이 발생하면 accept가 발생한다.
    # 그럼 client 소켓과 addr(주소)를 튜플로 받는다.
    client_socket, addr = server_socket.accept()
    th = threading.Thread(target=binder, args = (client_socket,addr))
    # 쓰레드를 이용해서 client 접속 대기를 만들고 다시 accept로 넘어가서 다른 client를 대기한다.
    th.start();
except:
    print("server")
finally:
    server_socket.close()

Connected by ('127.0.0.1', 58907)
Received from ('127.0.0.1', 58907) 이른 아침 작은 새들 노랫소리 들려오면 
언제나 그랬듯 아쉽게 잠을 깬다 
창문 하나 햇살 가득 눈부시게 비쳐오고 
서늘한 냉기에 재채기할까 말까
메시지 확인 이른 아침 작은 새들 노랫소리 들려오면 
언제나 그랬듯 아쉽게 잠을 깬다 
창문 하나 햇살 가득 눈부시게 비쳐오고 
서늘한 냉기에 재채기할까 말까
긍정 93% 입니다
긍정 93% 입니다
긍정 93% 입니다
Received from ('127.0.0.1', 58907) 
메시지 확인 
부정 -45% 입니다
부정 -45% 입니다
부정 -45% 입니다
except :  ('127.0.0.1', 58907)
Connected by ('127.0.0.1', 62598)
Received from ('127.0.0.1', 62598) 이른 아침 작은 새들 노랫소리 들려오면 
언제나 그랬듯 아쉽게 잠을 깬다 
창문 하나 햇살 가득 눈부시게 비쳐오고 
서늘한 냉기에 재채기할까 말까
메시지 확인 이른 아침 작은 새들 노랫소리 들려오면 
언제나 그랬듯 아쉽게 잠을 깬다 
창문 하나 햇살 가득 눈부시게 비쳐오고 
서늘한 냉기에 재채기할까 말까
긍정 93% 입니다
긍정 93% 입니다
긍정 93% 입니다
Received from ('127.0.0.1', 62598) 
메시지 확인 
부정 -45% 입니다
부정 -45% 입니다
부정 -45% 입니다
except :  ('127.0.0.1', 62598)
